In [8]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import time
from tqdm import tqdm

DCA

In [66]:
# f^t = cx+ρ(|z|_1)-zs_z^{t-1}の計算
def solve_main(T, C, M, xi, rho, s_z_t_minus_1):
    '''
    DCアルゴリズムによる最適化問題の解法
    K：供給者数
    J：需要者数
    n：シナリオ数
    '''

    K, J = C.shape
    n = len(xi) 

    # モデルの作成
    model = gp.Model("dca")

    # 変数の設定
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name="X")
    z = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="z")

    # 目的関数の設定
    objective = gp.quicksum(C[k, j] * X[k, j] for k in range(K) for j in range(J)) + rho * gp.quicksum(z[t] for t in range(n)) - gp.quicksum(z[t] * s_z_t_minus_1[t] for t in range(n))
    model.setObjective(objective, GRB.MINIMIZE)

    # 制約条件の設定
    for i in range(n):
        model.addConstr(T @ X >= (1 - z[i]) * np.array(list(xi[i])))

    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])

    # 時間上限の設定
    model.Params.TimeLimit = 300

    # 最適化
    model.optimize()

    # 結果の取得
    if model.status == GRB.OPTIMAL:
        optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)])
        optimal_z = np.array([z[i].x for i in range(n)])
        optimal_value = model.objVal
        return optimal_X, optimal_z, optimal_value
    else:
        print("最適解が得られませんでした")
        return None, None, None

In [26]:
def process_vector(w, p):
    w = np.asarray(w, dtype=float)
    # wを減少順にソート
    sorted_indices = np.argsort(w)[::-1]
    sorted_w = np.abs(w[sorted_indices])

    # w(i)をp番目まで１, それ以外を0にする
    w = np.zeros_like(w)
    w[sorted_indices[:p]] = 1
    return sorted_w, w

In [20]:
def dca(T, C, M, xi, rho, epsilon, z_t_minus_1, varepsilon=0.001, max_iteration=100):
    '''
    DCアルゴリズムを解く
    '''
    
    # 初期値の設定
    f_t_minus_1 = 0
    for iteration in range(max_iteration):
        print("反復回数：", iteration+1, "回")
        # 劣微分の計算
        _, s_z_t_minus_1 = process_vector(z_t_minus_1, int(len(xi)*epsilon))

        # 凸最適化問題を解く
        X, z, f_t = solve_main(T, C, M, xi, rho, s_z_t_minus_1)
        print("X_t:", X)
        print("z_t:", z)
        print("f_t:", f_t)

        if np.abs(f_t - f_t_minus_1) < varepsilon:
            break

        # 次のイテレーションのための準備
        f_t_minus_1 = f_t
        z_t_minus_1 = z

In [40]:
# パラメータの設定
# 供給者数
K = 3
# 需要者数
J = 3
# シナリオ数
n = 10
# 重み
T = np.ones((1, K))
# コスト
C = np.random.uniform(10, 50, size=(K, J))
# 供給量
M = np.random.randint(10, 20, size=K)

# シナリオの設定
xi = []
making_xi_time_s = time.time()
for _ in range(n):
    mean = np.random.uniform(0, 10, J)  # 平均ベクトルをランダムに生成
    variance = np.random.uniform(1, 5, J)  # 分散ベクトルをランダムに生成
    consumer_demand = np.abs(np.random.normal(mean, np.sqrt(variance), J))  # 正規分布に従うランダムベクトルを生成
    xi.append(consumer_demand)

# パラメータの設定
rho = 10000
epsilon = 0.1
z_t_minus_1 = np.random.rand(n)
varepsilon = 1
max_iteration = 100


In [7]:
# seed値の設定
np.random.seed(0)

In [39]:
print(T)

[1.]


In [19]:
print(C)

[[27.48127815 37.90524784 12.40901887]
 [36.67066862 36.82551478 18.41530244]
 [15.15705191 22.61713404 24.54843084]]


In [41]:
dca(T, C, M, xi, rho, epsilon, z_t_minus_1, varepsilon, max_iteration)

反復回数： 1 回
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 33 rows, 19 columns and 129 nonzeros
Model fingerprint: 0x9e42e828
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+01]
Presolve time: 0.00s
Presolved: 33 rows, 19 columns, 129 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.569253e+01   0.000000e+00      0s
       7    4.5107565e+02   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.510756499e+02
X_t: [[ 0.          8.93834146  0.        ]
 [ 0.          0.         10.92406925]
 [ 9.36530382  1.06165854  0.        ]]
z_t: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
f_t

In [37]:
print(z_t_minus_1)

[0.64626444 0.21488074 0.18645822 0.80758027 0.74707947 0.67484735
 0.27689375 0.17490887 0.70447426 0.4631502 ]


In [35]:
print(xi)

[array([1.66482336, 3.59142794, 5.18836702]), array([4.89899584, 1.17106457, 0.45994694]), array([4.47251097, 2.03188984, 7.0083378 ]), array([3.88126314, 7.54639256, 9.44127502]), array([ 7.10664814, 11.00868072,  1.35771662]), array([0.68301587, 5.65919948, 3.72121428]), array([12.39914639,  9.25188287,  9.23477906]), array([ 5.64673466,  4.92510699, 10.61996379]), array([2.5387455 , 8.38042646, 3.14883174]), array([2.10785693, 9.61050087, 1.7361977 ])]


PMIP

In [44]:
def solve_pmip(T, C, M, epsilon, xi):
    '''
    PMIPアルゴリズムによる最適化問題の解法
    K：供給者数
    J：需要者数
    n：シナリオ数
    '''

    K, J = C.shape
    n = len(xi) 

    # モデルの作成
    model = gp.Model("pmip")

    # 変数の設定
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name="X")
    z = model.addVars(n, lb=0.0, vtype=GRB.BINARY, name="z")

    # 目的関数の設定
    objective = gp.quicksum(C[k, j] * X[k, j] for k in range(K) for j in range(J))
    model.setObjective(objective, GRB.MINIMIZE)

    # 制約条件の設定
    for i in range(n):
        model.addConstr(T @ X >= (1 - z[i]) * np.array(list(xi[i])))
    
    model.addConstr(gp.quicksum(z[t] for t in range(n)) <= n * epsilon)

    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])

    # 時間上限の設定
    model.Params.TimeLimit = 300

    # 最適化
    model.optimize()

    # 結果の取得
    if model.status == GRB.OPTIMAL:
        optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)])
        optimal_z = np.array([z[i].x for i in range(n)])
        optimal_value = model.objVal
        return optimal_X, optimal_z, optimal_value
    else:
        print("最適解が得られませんでした")
        return None, None, None

In [45]:
solve_pmip(T, C, M, epsilon, xi)

Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 34 rows, 19 columns and 139 nonzeros
Model fingerprint: 0x18e0bc68
Variable types: 9 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+01]
Presolve time: 0.00s
Presolved: 34 rows, 19 columns, 139 nonzeros
Variable types: 9 continuous, 10 integer (10 binary)
Found heuristic solution: objective 451.0756499

Root relaxation: objective 3.051303e+02, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  305.13027    0    5  451.07565  305.13027  32.4%    

(array([[0.        , 8.93834146, 0.        ],
        [0.        , 0.        , 7.96521854],
        [8.29765861, 1.06165854, 0.        ]]),
 array([-0.,  1.,  0., -0., -0., -0., -0., -0., -0., -0.]),
 392.8068596557003)

In [46]:
optimal_X, optimal_z, optimal_value = dca(T, C, M, xi, rho, epsilon, z_t_minus_1, varepsilon, max_iteration)
print("Optimal X:", optimal_X)
print("Optimal z:", optimal_z)
print("Optimal value:", optimal_value)


反復回数： 1 回
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 33 rows, 19 columns and 129 nonzeros
Model fingerprint: 0x9e42e828
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+01]
Presolve time: 0.04s
Presolved: 33 rows, 19 columns, 129 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.569253e+01   0.000000e+00      0s
       7    4.5107565e+02   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.05 seconds (0.00 work units)
Optimal objective  4.510756499e+02
X_t: [[ 0.          8.93834146  0.        ]
 [ 0.          0.         10.92406925]
 [ 9.36530382  1.06165854  0.        ]]
z_t: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
f_t

TypeError: cannot unpack non-iterable NoneType object

In [50]:
X_t = [[ 0,        8.93834146, 0.        ],
 [ 0,    0,  10.92406925],
 [ 9.36530382,  1.06165854,  0.        ]]

In [51]:
print(X_t)

[[0, 8.93834146, 0.0], [0, 0, 10.92406925], [9.36530382, 1.06165854, 0.0]]


In [52]:
C @ X_t

array([[155.4486349 , 407.4866875 , 198.75940342],
       [105.06108896, 145.92524575, 424.76305891],
       [175.1126354 , 376.30991041, 149.7017693 ]])

In [55]:
sum(C @ X_t)

array([435.62235925, 929.72184366, 773.22423163])

In [53]:
X_t_mip = [[0.        , 8.93834146, 0.        ],
        [0.        , 0.        , 7.96521854],
        [8.29765861, 1.06165854, 0.        ]]

In [54]:
C @ X_t_mip

array([[137.72748098, 407.4866875 , 144.92420809],
       [ 93.08411837, 145.92524575, 309.71339658],
       [155.14978422, 376.30991041, 109.15413305]])

In [56]:
sum(C @ X_t_mip)

array([385.96138357, 929.72184366, 563.79173771])

In [59]:
objective_value = np.sum(C @ X_t)
print("Objective Function Value:", objective_value)


Objective Function Value: 2138.568434537703


In [60]:
objective_value = np.sum(C @ X_t_mip)
print("MIP Objective Function Value:", objective_value)

MIP Objective Function Value: 1879.4749649427656


In [64]:
# f^t = cx+ρ(|z|_1)-zs_z^{t-1}の計算
def solve_main(T, C, M, xi, rho, s_z_t_minus_1):
    '''
    DCアルゴリズムによる最適化問題の解法
    K：供給者数
    J：需要者数
    n：シナリオ数
    '''

    K, J = C.shape
    n = len(xi) 

    # モデルの作成
    model = gp.Model("dca")

    # 変数の設定
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name="X")
    z = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="z")

    # 目的関数の設定
    objective = gp.quicksum(np.sum(C @ X)) + rho * gp.quicksum(z[t] for t in range(n)) - gp.quicksum(z[t] * s_z_t_minus_1[t] for t in range(n))
    model.setObjective(objective, GRB.MINIMIZE)

    # 制約条件の設定
    for i in range(n):
        model.addConstr(T @ X >= (1 - z[i]) * np.array(list(xi[i])))

    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])

    # 時間上限の設定
    model.Params.TimeLimit = 300

    # 最適化
    model.optimize()

    # 結果の取得
    if model.status == GRB.OPTIMAL:
        optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)])
        optimal_z = np.array([z[i].x for i in range(n)])
        optimal_value = model.objVal
        return optimal_X, optimal_z, optimal_value
    else:
        print("最適解が得られませんでした")
        return None, None, None

In [67]:
dca(T, C, M, xi, rho, epsilon, z_t_minus_1, varepsilon, max_iteration)

反復回数： 1 回
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 33 rows, 19 columns and 129 nonzeros
Model fingerprint: 0x9e42e828
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+01]
Presolve time: 0.00s
Presolved: 33 rows, 19 columns, 129 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.569253e+01   0.000000e+00      0s
       7    4.5107565e+02   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.00 seconds (0.00 work units)
Optimal objective  4.510756499e+02
X_t: [[ 0.          8.93834146  0.        ]
 [ 0.          0.         10.92406925]
 [ 9.36530382  1.06165854  0.        ]]
z_t: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
f_t

PMIP_STAR

In [ ]:
def solve_mip_star(T, C, M, epsilon, xi):
    K, J = C.shape
    n = len(xi)
    p = int(n * epsilon + 1)
    h = xi[np.argsort(np.sum(xi, axis=1))[::-1]]



In [ ]:
import numpy as np
import numpy as np

def calculate_subgradients(f, x, epsilon=1e-6):
    """
    Calculate subgradients of a function f at point x.
    f: function to calculate subgradients for
    x: point at which to calculate subgradients
    epsilon: small value for numerical differentiation
    Returns: subgradients of f at x
    """
    subgradients = []
    n = len(x)
    for i in range(n):
        delta = np.zeros(n)
        delta[i] = epsilon
        subgradient = (f(x + delta) - f(x - delta)) / (2 * epsilon)
        subgradients.append(subgradient)
    return subgradients
# 配列の定義
arr = np.array([5, 2, 9, 1, 7])

# argmaxの計算
max_index = np.argmax(arr)

print("argmax:", max_index)
print("最大値:", arr[max_index])


In [71]:
import numpy as np
from itertools import combinations

def calculate_argmax(w, K):
    n = len(w)
    max_value = float('-inf')
    max_combination = None

    for indices in combinations(range(n), K):
        s = np.zeros(n)
        s[list(indices)] = 1
        value = np.dot(w, s)
        print("s:", s)
        print("value:", value)
        
        if value > max_value:
            max_value = value
            max_combination = s
    
    return max_combination

w = np.array([1, 2, 3, 4, 5])
K = 3

argmax_s = calculate_argmax(w, K)
print("argmax_s:", argmax_s)


s: [1. 1. 1. 0. 0.]
value: 6.0
s: [1. 1. 0. 1. 0.]
value: 7.0
s: [1. 1. 0. 0. 1.]
value: 8.0
s: [1. 0. 1. 1. 0.]
value: 8.0
s: [1. 0. 1. 0. 1.]
value: 9.0
s: [1. 0. 0. 1. 1.]
value: 10.0
s: [0. 1. 1. 1. 0.]
value: 9.0
s: [0. 1. 1. 0. 1.]
value: 10.0
s: [0. 1. 0. 1. 1.]
value: 11.0
s: [0. 0. 1. 1. 1.]
value: 12.0
argmax_s: [0. 0. 1. 1. 1.]


In [72]:
def sort_vector_desc(vector):
    sorted_vector = np.sort(vector)[::-1]
    return sorted_vector

# テスト用のベクトル
vector = np.array([3, 1, 4, 2, 5])

# ベクトルを降順に並び替える
sorted_vector = sort_vector_desc(vector)

print(sorted_vector)


[5 4 3 2 1]


In [69]:
def process_vector(w, K):
    w = np.asarray(w, dtype=float)
    # wを減少順に並べ替える
    sorted_indices = np.argsort(-np.abs(w))
    sorted_w = np.abs(w[sorted_indices])
    # print("sorted indices", sorted_indices)
    # print("sorted_w", sorted_w)

    # w(i)をK番目まで1、それ以外を0にする
    s = np.zeros_like(w)
    s[sorted_indices[:K]] = np.sign(w[sorted_indices[:K]])
    # print("s", s)

    return sorted_w, s

process_vector(vector, 3)

(array([5., 4., 3., 2., 1.]), array([1., 0., 1., 0., 1.]))

In [73]:
import numpy as np

w = np.array([1, 2, 3, 4, 5])
K = 3

# argmaxの計算
max_indices = np.argsort(-w)[:K]
argmax_s = np.zeros_like(w)
argmax_s[max_indices] = 1

print("argmax_s:", argmax_s)

argmax_s: [0 0 1 1 1]
